In [1]:
from warnings import filterwarnings
filterwarnings('ignore')
# Import necessary libraries
import yfinance as yf  # Library to fetch stock data from Yahoo Finance
import pandas as pd  # Library for data manipulation and analysis
import numpy as np  # Library for numerical computing
import plotly.graph_objs as go  # Library for interactive plotting
import smtplib  # Library for sending email
from email.mime.text import MIMEText  # MIMEText to create email body
from email.mime.multipart import MIMEMultipart  # MIMEMultipart to create email message
from datetime import datetime  # Library for datetime operations
import time  # Library for time-related operations
from IPython.display import clear_output, display  # Library for clearing output and displaying output
import subprocess  # Library for subprocess management
import re  # Library for regular expressions
import pyautogui  # Library for GUI automation

from plotly.subplots import make_subplots


In [2]:
# Function to send email messages

def send_email(message_text):
    """
    Function to send email messages.

    Parameters:
    - message_text: Text message to be sent via email.

    The function sets up email parameters, creates a MIME multipart message,
    and sends the email using the smtplib library to the specified recipient.
    """

    # Set up email parameters
    sender_email = "kaushal.cilans@gmail.com"  # Email address of the sender
    receiver_email = "kaushaljadav111@gmail.com"  # Email address of the receiver
    password = "fndkdayybiqfotsy"  # Password of the sender's email account

    subject = "EMA Crossover Alert"  # Subject of the email
    body = message_text  # Body of the email, contains the message text

    # Create message
    message = MIMEMultipart()  # Create a MIME multipart message
    message["From"] = sender_email  # Set the 'From' field of the email
    message["To"] = receiver_email  # Set the 'To' field of the email
    message["Subject"] = subject  # Set the subject of the email
    message.attach(MIMEText(body, "plain"))  # Attach the body of the email as plain text

    # Print Statement Before Sending Email
    print("Sending Email...")  # Print statement to indicate email sending process is initiated

    # Establish a connection with the SMTP server
    try:
        with smtplib.SMTP("smtp.gmail.com", 587) as server:  # Connect to Gmail SMTP server
            server.starttls()  # Start TLS encryption
            server.login(sender_email, password)  # Login to the email account
            server.sendmail(sender_email, receiver_email, message.as_string())  # Send the email message
        print("Email Sent Successfully!")  # Print statement if email is sent successfully
    except Exception as e:
        print("Error Sending Email:", str(e))  # Print statement if an error occurs while sending the email


In [3]:
# Function to send WhatsApp messages to numbers listed in a CSV file

def send_whatsapp_messages(numbers_filename, message_text):
    """
    Function to send WhatsApp messages to numbers listed in a CSV file.

    Parameters:
    - numbers_filename: Path to the CSV file containing contact numbers.
    - message_text: Text message to be sent via WhatsApp.

    The function reads the contact numbers from the provided CSV file, validates each number,
    and sends the specified message via WhatsApp using subprocess and pyautogui libraries.
    """

    # Display the selected contact file path
    print("\n========================= Selected Contact File Path ===========================\n")
    print(numbers_filename)

    # Read the CSV file containing contact numbers into a DataFrame
    df1 = pd.read_csv(numbers_filename)
    print(df1)

    # Process the input text message
    input_txt = message_text

    # List to store rejected numbers
    rejected_number = []

    # Iterate through each row in the DataFrame
    for _, i in df1.iterrows():
        print(i['Number'])
        print(i['Name'])

        # Check if the number matches the valid pattern
        r = re.fullmatch('[6-9][0-9]{9}', str(i['Number']))
        if r is not None:
            print('Valid Number')

            raw_text = "Hello there, *{name}* !".format(name=i['Name'])
            text = raw_text.replace(" ", "%20")
            input_txt = input_txt.replace(' ', '%20')
            input_txt = input_txt.replace('\n', '%0A')
            input_txt = input_txt.replace(':)', '\U0001F601')

            print(input_txt)

            try:
                print('Opening WhatsApp...')
                subprocess.Popen(["cmd", "/C", "start whatsapp://"], shell=True)
                time.sleep(2)
                subprocess.Popen(["cmd", "/C", "start whatsapp://send?phone=" + str(i['Number'])], shell=True)
                time.sleep(2)
                subprocess.Popen(["cmd", "/C", "start whatsapp://send?phone=" + str(i['Number']) + "^&text=" + input_txt], shell=True)
                time.sleep(2)
                print('WhatsApp Opened')
                pyautogui.click(1000,1000)
                pyautogui.press('enter')  # Press Enter key to send message
                print('Message Sent')

            except Exception as e:
                print(e)

        else:
            print('Not a valid number')
            rejected_number.append(str(i['Number']))

    # Print the list of rejected numbers
    print(rejected_number)

    # Create a DataFrame for rejected numbers and store them in a CSV file
    rejected_num = pd.DataFrame()
    for i in rejected_number:
        print(i)
        rejected = df1.loc[df1['Number'] == i]
        rejected_num = pd.concat([rejected_num, rejected])
        print(rejected_num)

    rejected_num.to_csv('rejected_numbers.csv')


In [4]:
# Initialize figure
fig = go.Figure()

In [5]:
# def MACD_plot(data, fig):
    
#     """
#     Function to plot MACD and Signal Line and send WhatsApp alerts based on crossovers.

#     Parameters:
#     - data: DataFrame containing financial market data with 'Datetime' and 'Close' columns.
#     - fig: Plotly figure to update and display.

#     The function calculates the 12 and 26-period Exponential Moving Averages (EMA), MACD, and Signal Line.
#     It then checks for MACD crossovers and sends WhatsApp alerts using the send_whatsapp_messages function.
#     Finally, it updates the plot with MACD and Signal Line and displays the figure.

#     Note: This function relies on the send_whatsapp_messages and send_email functions for alert notifications.
#     """

#     # Define short and long windows for EMA calculation
#     short_window = 13
#     large_window = 34

#     # Calculate the 12-period Exponential Moving Average (EMA)
#     data['EMA_short_5m'] = data['5m_Close'].ewm(span=short_window, adjust=False).mean()

#     # Calculate the 26-period Exponential Moving Average (EMA)
#     data['EMA_large_5m'] = data['5m_Close'].ewm(span=large_window, adjust=False).mean()

#     # Calculate the Moving Average Convergence Divergence (MACD) line
#     data['MACD_5m'] = data['EMA_short_5m'] - data['EMA_large_5m']

#     # Calculate the 9-period EMA of MACD (Signal Line)
#     data['Signal_Line_5m'] = data['MACD_5m'].ewm(span=9, adjust=False).mean()
    
#     data['msg_signal_5m'] = np.where(data['MACD_5m'] > data['Signal_Line_5m'].shift(1), 1, 0)



#     # Calculate the 12-period Exponential Moving Average (EMA)
#     data['EMA_short_15m'] = data['15m_Close'].ewm(span=short_window, adjust=False).mean()

#     # Calculate the 26-period Exponential Moving Average (EMA)
#     data['EMA_large_15m'] = data['15m_Close'].ewm(span=large_window, adjust=False).mean()

#     # Calculate the Moving Average Convergence Divergence (MACD) line
#     data['MACD_15m'] = data['EMA_short_15m'] - data['EMA_large_15m']

#     # Calculate the 9-period EMA of MACD (Signal Line)
#     data['Signal_Line_15m'] = data['MACD_15m'].ewm(span=9, adjust=False).mean()

#     data['msg_signal_15m'] = np.where(data['MACD_15m'] > data['Signal_Line_15m'].shift(1), 1, 0)




#         # Calculate the 12-period Exponential Moving Average (EMA)
#     data['EMA_short_60m'] = data['60m_Close'].ewm(span=short_window, adjust=False).mean()

#     # Calculate the 26-period Exponential Moving Average (EMA)
#     data['EMA_large_60m'] = data['60m_Close'].ewm(span=large_window, adjust=False).mean()

#     # Calculate the Moving Average Convergence Divergence (MACD) line
#     data['MACD_60m'] = data['EMA_short_60m'] - data['EMA_large_60m']

#     # Calculate the 9-period EMA of MACD (Signal Line)
#     data['Signal_Line_60m'] = data['MACD_60m'].ewm(span=9, adjust=False).mean()

#     data['msg_signal_60m'] = np.where(data['MACD_60m'] > data['Signal_Line_60m'].shift(1), 1, 0)



    
#     # Check for MACD and Signal Line crossovers in the last two rows in 5m timme interval than 15min and than after 60m time interval and in the basis of the combination in icreasing order send the customize message to the user
#     if ((data['msg_signal_5m'].iloc[-1] == 1) & (data['msg_signal_5m'].iloc[-2] == 0)).all():
#         message = "MACD Crossover Alert: Cross Above Signal Line in 5m time interval"
#         send_whatsapp_messages("leads.csv", message)  # Adjust the filename accordingly
#         send_email(message_text=message)  # Send a detailed email
    
#     elif ((data['msg_signal_5m'].iloc[-1] == 0) & (data['msg_signal_5m'].iloc[-2] == 1)).all():
#         message = "MACD Crossover Alert: Cross Below Signal Line in 5m time interval"
#         send_whatsapp_messages("leads.csv", message)  # Adjust the filename accordingly
#         send_email(message_text=message)  # Send a detailed email
        
#     else:
#         print("No significant crossover detected!")

      

#     if ((data['msg_signal_15m'].iloc[-1] == 0) & (data['msg_signal_15m'].iloc[-2] == 1) & (data['msg_signal_5m'].iloc[-1] == 0)).all():
#         message = "MACD Crossover Alert: The Cross over to below signal line in 15 minutes time inetrval and is continued from 5 minutes time frame as well"
#         send_whatsapp_messages("leads.csv", message)
#         send_email(message_text=message)

#     elif ((data['msg_signal_15m'].iloc[-1] == 0) & (data['msg_signal_15m'].iloc[-2] == 1)).all():
#         message = "MACD Crossover Alert: Cross Below Signal Line in 15m time interval"
#         send_whatsapp_messages("leads.csv", message)
#         send_email(message_text=message)


#     elif ((data['msg_signal_15m'].iloc[-1] == 1) & (data['msg_signal_15m'].iloc[-2] == 0) & (data['msg_signal_5m'].iloc[-1] == 1)).all():
#         message = "MACD Crossover Alert: The Cross over to above signal line in 15 minutes time inetrval and is continued from 5 minutes time frame as well"
#         send_whatsapp_messages("leads.csv", message)
#         send_email(message_text=message)

#     elif ((data['msg_signal_15m'].iloc[-1] == 1) & (data['msg_signal_15m'].iloc[-2] == 0)).all():
#         message = "MACD Crossover Alert: Cross Above Signal Line in 15m time interval"
#         send_whatsapp_messages("leads.csv", message)
#         send_email(message_text=message)

#     else:
#         print("No significant crossover detected!")
        
#     if ((data['msg_signal_60m'].iloc[-1] == 0) & (data['msg_signal_60m'].iloc[-2] == 1) & (data['msg_signal_15m'].iloc[-1] == 0)).all():
#         message = "MACD Crossover Alert: The Cross over to below signal line in 60m time interval and is continued from 15 minutes time frame as well"
#         send_whatsapp_messages("leads.csv", message)
#         send_email(message_text=message)

#     elif ((data['msg_signal_60m'].iloc[-1] == 0) & (data['msg_signal_60m'].iloc[-2] == 1)).all():
#         message = "MACD Crossover Alert: Cross Below Signal Line in 60m time interval"
#         send_whatsapp_messages("leads.csv", message)
#         send_email(message_text=message)


#     elif ((data['msg_signal_60m'].iloc[-1] == 1) & (data['msg_signal_60m'].iloc[-2] == 0) & (data['msg_signal_15m'].iloc[-1] == 1)).all():
#         message = "MACD Crossover Alert: The Cross over to above signal line in 60m time interval and is continued from 15 minutes time frame as well"
#         send_whatsapp_messages("leads.csv", message)
#         send_email(message_text=message)

#     elif ((data['msg_signal_60m'].iloc[-1] == 1) & (data['msg_signal_60m'].iloc[-2] == 0)).all():
#         message = "MACD Crossover Alert: Cross Above Signal Line in 60m time interval"
#         send_whatsapp_messages("leads.csv", message)
#         send_email(message_text=message)

#     else:
#         print("No significant crossover detected!")

#     # Update traces for MACD and Signal Line
#     trace_macd_5m = go.Scatter(x=data['Datetime'].tail(50), y=data['MACD_5m'].tail(50), mode='lines', name='MACD (5m)')
#     trace_signal_line_5m = go.Scatter(x=data['Datetime'].tail(50), y=data['Signal_Line_5m'].tail(50), mode='lines', name='Signal Line (5m)')

#     trace_macd_15m = go.Scatter(x=data['Datetime'].tail(50), y=data['MACD_15m'].tail(50), mode='lines', name='MACD (15m)')
#     trace_signal_line_15m = go.Scatter(x=data['Datetime'].tail(50), y=data['Signal_Line_15m'].tail(50), mode='lines', name='Signal Line (15m)')

#     trace_macd_60m = go.Scatter(x=data['Datetime'].tail(50), y=data['MACD_60m'].tail(50), mode='lines', name='MACD (60m)')
#     trace_signal_line_60m = go.Scatter(x=data['Datetime'].tail(50), y=data['Signal_Line_60m'].tail(25), mode='lines', name='Signal Line (60m)')

#     # Update layout
#     layout = go.Layout(title='MACD', yaxis=dict(title='Value'),width=1200, height=1000)

#     # Create subplots
#     fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.1)

#     # Add traces to subplots
#     fig.add_trace(trace_macd_5m, row=1, col=1)
#     fig.add_trace(trace_signal_line_5m, row=1, col=1)

#     fig.add_trace(trace_macd_15m, row=2, col=1)
#     fig.add_trace(trace_signal_line_15m, row=2, col=1)

#     fig.add_trace(trace_macd_60m, row=3, col=1)
#     fig.add_trace(trace_signal_line_60m, row=3, col=1)

#     # Add headers
#     fig.add_annotation(text="5min", xref="paper", yref="paper", x=0.0, y=1.03, showarrow=False, font=dict(size=14))
#     fig.add_annotation(text="15min", xref="paper", yref="paper", x=0.0, y=0.65, showarrow=False, font=dict(size=14))
#     fig.add_annotation(text="60min", xref="paper", yref="paper", x=0.0, y=0.27, showarrow=False, font=dict(size=14))

#     # Update layout
#     fig.update_layout(layout)

#     # Display the figure
#     clear_output(wait=True)
#     display(fig)

#     # Print the last two rows of data
#     # print(data.iloc[-5:])

#     return data, fig




def MACD_plot(data, fig, short_window, large_window):
    
    """
    Function to plot MACD and Signal Line and send WhatsApp alerts based on crossovers.

    Parameters:
    - data: DataFrame containing financial market data with 'Datetime' and 'Close' columns.
    - fig: Plotly figure to update and display.

    The function calculates the 12 and 26-period Exponential Moving Averages (EMA), MACD, and Signal Line.
    It then checks for MACD crossovers and sends WhatsApp alerts using the send_whatsapp_messages function.
    Finally, it updates the plot with MACD and Signal Line and displays the figure.

    Note: This function relies on the send_whatsapp_messages and send_email functions for alert notifications.
    """

    # Define short and long windows for EMA calculation
    short_window = short_window
    large_window = large_window

    # Calculate the 12-period Exponential Moving Average (EMA)
    data['EMA_short_5m'] = data['5m_Close'].ewm(span=short_window, adjust=False).mean()

    # Calculate the 26-period Exponential Moving Average (EMA)
    data['EMA_large_5m'] = data['5m_Close'].ewm(span=large_window, adjust=False).mean()

    # Calculate the Moving Average Convergence Divergence (MACD) line
    data['MACD_5m'] = data['EMA_short_5m'] - data['EMA_large_5m']

    # Calculate the 9-period EMA of MACD (Signal Line)
    data['Signal_Line_5m'] = data['MACD_5m'].ewm(span=9, adjust=False).mean()
    
    data['msg_signal_5m'] = np.where(data['MACD_5m'] > data['Signal_Line_5m'].shift(1), 1, 0)



    # Calculate the 12-period Exponential Moving Average (EMA)
    data['EMA_short_15m'] = data['15m_Close'].ewm(span=short_window, adjust=False).mean()

    # Calculate the 26-period Exponential Moving Average (EMA)
    data['EMA_large_15m'] = data['15m_Close'].ewm(span=large_window, adjust=False).mean()

    # Calculate the Moving Average Convergence Divergence (MACD) line
    data['MACD_15m'] = data['EMA_short_15m'] - data['EMA_large_15m']

    # Calculate the 9-period EMA of MACD (Signal Line)
    data['Signal_Line_15m'] = data['MACD_15m'].ewm(span=9, adjust=False).mean()

    data['msg_signal_15m'] = np.where(data['MACD_15m'] > data['Signal_Line_15m'].shift(1), 1, 0)




        # Calculate the 12-period Exponential Moving Average (EMA)
    data['EMA_short_60m'] = data['60m_Close'].ewm(span=short_window, adjust=False).mean()

    # Calculate the 26-period Exponential Moving Average (EMA)
    data['EMA_large_60m'] = data['60m_Close'].ewm(span=large_window, adjust=False).mean()

    # Calculate the Moving Average Convergence Divergence (MACD) line
    data['MACD_60m'] = data['EMA_short_60m'] - data['EMA_large_60m']

    # Calculate the 9-period EMA of MACD (Signal Line)
    data['Signal_Line_60m'] = data['MACD_60m'].ewm(span=9, adjust=False).mean()

    data['msg_signal_60m'] = np.where(data['MACD_60m'] > data['Signal_Line_60m'].shift(1), 1, 0)





    # Calculate the 12-period Exponential Moving Average (EMA)
    data['EMA_short_1d'] = data['1d_Close'].ewm(span=short_window, adjust=False).mean()

    # Calculate the 26-period Exponential Moving Average (EMA)
    data['EMA_large_1d'] = data['1d_Close'].ewm(span=large_window, adjust=False).mean()

    # Calculate the Moving Average Convergence Divergence (MACD) line
    data['MACD_1d'] = data['EMA_short_1d'] - data['EMA_large_1d']

    # Calculate the 9-period EMA of MACD (Signal Line)
    data['Signal_Line_1d'] = data['MACD_1d'].ewm(span=9, adjust=False).mean()

    data['msg_signal_1d'] = np.where(data['MACD_1d'] > data['Signal_Line_1d'].shift(1), 1, 0)





    if ((data['msg_signal_1d'].iloc[-1] == 0) & (data['msg_signal_1d'].iloc[-2] == 1) & (data['msg_signal_60m'].iloc[-1] == 0) & (data['msg_signal_15m'].iloc[-1] == 0)).all():
        message = f"MACD Crossover Alert: The Cross over to below signal line in 1 day time interval and is continued from 60 minutes and 15 minutes time frame as well"
        # send_whatsapp_messages("leads.csv", message)
        # send_email(message_text=message)

    elif ((data['msg_signal_1d'].iloc[-1] == 0) & (data['msg_signal_1d'].iloc[-2] == 1) & (data['msg_signal_60m'].iloc[-1] == 0)).all():
        message = f"MACD Crossover Alert: Cross Below Signal Line in 1 day time interval and is continued from 60 minutes time frame as well"
        # send_whatsapp_messages("leads.csv", message)
        # send_email(message_text=message)

    elif ((data['msg_signal_1d'].iloc[-1] == 0) & (data['msg_signal_1d'].iloc[-2] == 1)).all():
        message = f"MACD Crossover Alert: Cross Below Signal Line in 1 day time interval"
        # send_whatsapp_messages("leads.csv", message)
        # send_email(message_text=message)


    elif ((data['msg_signal_1d'].iloc[-1] == 1) & (data['msg_signal_1d'].iloc[-2] == 0) & (data['msg_signal_60m'].iloc[-1] == 1) & (data['msg_signal_15m'].iloc[-1] == 1)).all():
        message = f"MACD Crossover Alert: The Cross over to above signal line in 1 day time interval and is continued from 60 minutes and 15 minutes time frame as well"
        # send_whatsapp_messages("leads.csv", message)
        # send_email(message_text=message)

    elif ((data['msg_signal_1d'].iloc[-1] == 1) & (data['msg_signal_1d'].iloc[-2] == 0) & (data['msg_signal_60m'].iloc[-1] == 1)).all():
        message = f"MACD Crossover Alert: The Cross over to above signal line in 1 day time interval and is continued from 60 minutes time frame as well"
        # send_whatsapp_messages("leads.csv", message)
        # send_email(message_text=message)

    elif ((data['msg_signal_1d'].iloc[-1] == 1) & (data['msg_signal_1d'].iloc[-2] == 0)).all():
        message = f"MACD Crossover Alert: Cross Above Signal Line in 1 day time interval"
        # send_whatsapp_messages("leads.csv", message)
        # send_email(message_text=message)

    else:
        print("No significant crossover detected in 1 day time interval!")


        
    if ((data['msg_signal_60m'].iloc[-1] == 0) & (data['msg_signal_60m'].iloc[-2] == 1) & (data['msg_signal_15m'].iloc[-1] == 0)).all():
        message = "MACD Crossover Alert: The Cross over to below signal line in 60m time interval and is continued from 15 minutes time frame as well"
        send_whatsapp_messages("leads.csv", message)
        send_email(message_text=message)

    elif ((data['msg_signal_60m'].iloc[-1] == 0) & (data['msg_signal_60m'].iloc[-2] == 1)).all():
        message = "MACD Crossover Alert: Cross Below Signal Line in 60m time interval"
        send_whatsapp_messages("leads.csv", message)
        send_email(message_text=message)


    elif ((data['msg_signal_60m'].iloc[-1] == 1) & (data['msg_signal_60m'].iloc[-2] == 0) & (data['msg_signal_15m'].iloc[-1] == 1)).all():
        message = "MACD Crossover Alert: The Cross over to above signal line in 60m time interval and is continued from 15 minutes time frame as well"
        send_whatsapp_messages("leads.csv", message)
        send_email(message_text=message)

    elif ((data['msg_signal_60m'].iloc[-1] == 1) & (data['msg_signal_60m'].iloc[-2] == 0)).all():
        message = "MACD Crossover Alert: Cross Above Signal Line in 60m time interval"
        send_whatsapp_messages("leads.csv", message)
        send_email(message_text=message)

    else:
        print("No significant crossover detected in 60m time interval!")



    if ((data['msg_signal_15m'].iloc[-1] == 0) & (data['msg_signal_15m'].iloc[-2] == 1) & (data['msg_signal_5m'].iloc[-1] == 0)).all():
        message = "MACD Crossover Alert: The Cross over to below signal line in 15 minutes time inetrval and is continued from 5 minutes time frame as well"
        send_whatsapp_messages("leads.csv", message)
        send_email(message_text=message)

    elif ((data['msg_signal_15m'].iloc[-1] == 0) & (data['msg_signal_15m'].iloc[-2] == 1)).all():
        message = "MACD Crossover Alert: Cross Below Signal Line in 15m time interval"
        send_whatsapp_messages("leads.csv", message)
        send_email(message_text=message)


    elif ((data['msg_signal_15m'].iloc[-1] == 1) & (data['msg_signal_15m'].iloc[-2] == 0) & (data['msg_signal_5m'].iloc[-1] == 1)).all():
        message = "MACD Crossover Alert: The Cross over to above signal line in 15 minutes time inetrval and is continued from 5 minutes time frame as well"
        send_whatsapp_messages("leads.csv", message)
        send_email(message_text=message)

    elif ((data['msg_signal_15m'].iloc[-1] == 1) & (data['msg_signal_15m'].iloc[-2] == 0)).all():
        message = "MACD Crossover Alert: Cross Above Signal Line in 15m time interval"
        send_whatsapp_messages("leads.csv", message)
        send_email(message_text=message)

    else:
        print("No significant crossover detected in 15m time interval!")


    
    # Check for MACD and Signal Line crossovers in the last two rows in 5m timme interval than 15min and than after 60m time interval and in the basis of the combination in icreasing order send the customize message to the user
    if ((data['msg_signal_5m'].iloc[-1] == 1) & (data['msg_signal_5m'].iloc[-2] == 0)).all():
        message = "MACD Crossover Alert: Cross Above Signal Line in 5m time interval"
        send_whatsapp_messages("leads.csv", message)  # Adjust the filename accordingly
        send_email(message_text=message)  # Send a detailed email
    
    elif ((data['msg_signal_5m'].iloc[-1] == 0) & (data['msg_signal_5m'].iloc[-2] == 1)).all():
        message = "MACD Crossover Alert: Cross Below Signal Line in 5m time interval"
        send_whatsapp_messages("leads.csv", message)  # Adjust the filename accordingly
        send_email(message_text=message)  # Send a detailed email
        
    else:
        print("No significant crossover detected in 5m time interval!")

      

    # Update traces for MACD and Signal Line
    trace_macd_5m = go.Scatter(x=data['Datetime'].tail(25), y=data['MACD_5m'].tail(25), mode='lines', name='MACD (5m)')
    trace_signal_line_5m = go.Scatter(x=data['Datetime'].tail(25), y=data['Signal_Line_5m'].tail(25), mode='lines', name='Signal Line (5m)')

    trace_macd_15m = go.Scatter(x=data['Datetime'].tail(25), y=data['MACD_15m'].tail(25), mode='lines', name='MACD (15m)')
    trace_signal_line_15m = go.Scatter(x=data['Datetime'].tail(25), y=data['Signal_Line_15m'].tail(25), mode='lines', name='Signal Line (15m)')

    trace_macd_60m = go.Scatter(x=data['Datetime'].tail(25), y=data['MACD_60m'].tail(25), mode='lines', name='MACD (60m)')
    trace_signal_line_60m = go.Scatter(x=data['Datetime'].tail(25), y=data['Signal_Line_60m'].tail(25), mode='lines', name='Signal Line (60m)')

    trace_macd_1d = go.Scatter(x=data['Datetime'].tail(25), y=data['MACD_1d'].tail(25), mode='lines', name='MACD (1d)')
    trace_signal_line_1d = go.Scatter(x=data['Datetime'].tail(25), y=data['Signal_Line_1d'].tail(25), mode='lines', name='Signal Line (1d)')


    # Update layout
    layout = go.Layout(title='MACD', yaxis=dict(title='Value'),width=1200, height=1333)

    # Create subplots
    fig = make_subplots(rows=4, cols=1, shared_xaxes=True, vertical_spacing=0.1)

    # Add traces to subplots
    fig.add_trace(trace_macd_5m, row=1, col=1)
    fig.add_trace(trace_signal_line_5m, row=1, col=1)

    fig.add_trace(trace_macd_15m, row=2, col=1)
    fig.add_trace(trace_signal_line_15m, row=2, col=1)

    fig.add_trace(trace_macd_60m, row=3, col=1)
    fig.add_trace(trace_signal_line_60m, row=3, col=1)

    fig.add_trace(trace_macd_1d, row=4, col=1)
    fig.add_trace(trace_signal_line_1d, row=4, col=1)

    # Add headers
    fig.add_annotation(text="5min", xref="paper", yref="paper", x=0.0, y=1.03, showarrow=False, font=dict(size=14))
    fig.add_annotation(text="15min", xref="paper", yref="paper", x=0.0, y=0.65, showarrow=False, font=dict(size=14))
    fig.add_annotation(text="60min", xref="paper", yref="paper", x=0.0, y=0.27, showarrow=False, font=dict(size=14))
    fig.add_annotation(text="1day", xref="paper", yref="paper", x=0.0, y=0.0, showarrow=False, font=dict(size=14))

    # Update layout
    fig.update_layout(layout)

    # Display the figure
    clear_output(wait=True)
    display(fig)

    # Print the last two rows of data
    # print(data.iloc[-5:])

    return data, fig


In [6]:
# def initial_fetch_data(symbol, date):

#     if symbol == "^NSEI" or symbol == "^NSEBANK":
#         symbol = symbol.upper()
#     else:
#         symbol = f"{symbol}.NS".upper()

#     date = datetime.strptime(date, '%Y-%m-%d')

#     # Download 5-minute data
#     min5 = yf.download(symbol, start=date, interval='5m')
#     min5.index = min5.index.strftime('%Y-%m-%d %H:%M:%S') 
#     min5.index = pd.to_datetime(min5.index)
#     min5['Date'] = min5.index.date
#     for i in range(len(min5.columns)):
#         min5.columns.values[i] = "5m_" + min5.columns.values[i]
#     min5.rename(columns={'5m_Date':'Date'}, inplace=True)
#     min5 = min5.reindex(columns=['Date','5m_Open', '5m_High', '5m_Low', '5m_Close', '5m_Adj Close', '5m_Volume'])

#     # Download 15-minute data
#     min15 = yf.download(symbol, start=date, interval='15m')
#     min15.index = min15.index.strftime('%Y-%m-%d %H:%M:%S')
#     min15.index = pd.to_datetime(min15.index)
#     min15['Date'] = min15.index.date
#     for i in range(len(min15.columns)):
#         min15.columns.values[i] = "15m_" + min15.columns.values[i]
#     min15.rename(columns={'15m_Date':'Date'}, inplace=True)
#     min15 = min15.reindex(columns=['Date','15m_Open', '15m_High', '15m_Low', '15m_Close', '15m_Adj Close', '15m_Volume'])
#     min15 = min15.resample('5min').ffill()

#     # Download 60-minute data
#     min60 = yf.download(symbol, start=date, interval='60m')
#     min60.index = min60.index.strftime('%Y-%m-%d %H:%M:%S')
#     min60.index = pd.to_datetime(min60.index)
#     min60['Date'] = min60.index.date
#     for i in range(len(min60.columns)):
#         min60.columns.values[i] = "60m_" + min60.columns.values[i]
#     min60.rename(columns={'60m_Date':'Date'}, inplace=True)
#     min60 = min60.reindex(columns=['Date','60m_Open', '60m_High', '60m_Low', '60m_Close', '60m_Adj Close', '60m_Volume'])
#     min60 = min60.resample('5min').ffill()

#     # Download 1-day data
#     day1 = yf.download(symbol, start=date, interval='1d')
#     day1.index = day1.index.strftime('%Y-%m-%d')
#     day1.index = pd.to_datetime(day1.index)
#     day1['Date'] = day1.index.date
#     for i in range(len(day1.columns)):
#         day1.columns.values[i] = "1d_" + day1.columns.values[i]
#     day1.rename(columns={'1d_Date':'Date'}, inplace=True)
#     day1 = day1.reindex(columns=['Date','1d_Open', '1d_High', '1d_Low', '1d_Close', '1d_Adj Close', '1d_Volume'])
#     day1 = day1.resample('5min').ffill()

#     # Concatenate dataframes
#     main_df = pd.concat([min5, min15, min60, day1], axis=1)
#     # Drop duplicate columns
#     main_df = main_df.loc[:, ~main_df.columns.duplicated()]
#     # pd.set_option('display.max_rows', None)
#     main_df = main_df.dropna(subset=['5m_Open','5m_High','5m_Low','5m_Close'])

#     # if symbol == "^NSEI" or symbol == "^NSEBANK":
#     #     main_df.to_sql(symbol, conn, if_exists='append', index=True, index_label='Datetime')
#     # else:
#     #     main_df.to_sql(symbol[:-3], conn, if_exists='append', index=True, index_label='Datetime')

#     return main_df

def fetch_data(symbol, date):

    if symbol == "^NSEI" or symbol == "^NSEBANK":
        symbol = symbol.upper()
    else:
        symbol = f"{symbol}.NS".upper()

    date = datetime.strptime(date, '%Y-%m-%d %H:%M:%S')
    # temp_end = datetime.strptime("2024-01-31", '%Y-%m-%d')

    # Download 5-minute data
    min5 = yf.download(symbol, start=date, interval='5m')
    min5.index = min5.index.strftime('%Y-%m-%d %H:%M:%S') 
    min5.index = pd.to_datetime(min5.index)
    min5['Date'] = min5.index.date
    for i in range(len(min5.columns)):
        min5.columns.values[i] = "5m_" + min5.columns.values[i]
    min5.rename(columns={'5m_Date':'Date'}, inplace=True)
    min5 = min5.reindex(columns=['Date','5m_Open', '5m_High', '5m_Low', '5m_Close', '5m_Adj Close', '5m_Volume'])

    # Download 15-minute data
    min15 = yf.download(symbol, start=date, interval='15m')
    min15.index = min15.index.strftime('%Y-%m-%d %H:%M:%S')
    min15.index = pd.to_datetime(min15.index)
    min15['Date'] = min15.index.date
    for i in range(len(min15.columns)):
        min15.columns.values[i] = "15m_" + min15.columns.values[i]
    min15.rename(columns={'15m_Date':'Date'}, inplace=True)
    min15 = min15.reindex(columns=['Date','15m_Open', '15m_High', '15m_Low', '15m_Close', '15m_Adj Close', '15m_Volume'])
    min15 = min15.resample('5T').ffill()

    # Download 60-minute data
    min60 = yf.download(symbol, start=date, interval='60m')
    min60.index = min60.index.strftime('%Y-%m-%d %H:%M:%S')
    min60.index = pd.to_datetime(min60.index)
    min60['Date'] = min60.index.date
    for i in range(len(min60.columns)):
        min60.columns.values[i] = "60m_" + min60.columns.values[i]
    min60.rename(columns={'60m_Date':'Date'}, inplace=True)
    min60 = min60.reindex(columns=['Date','60m_Open', '60m_High', '60m_Low', '60m_Close', '60m_Adj Close', '60m_Volume'])
    min60 = min60.resample('5T').ffill()

    # Download 60-minute data
    day1 = yf.download(symbol, start=date, interval='1d')
    day1.index = day1.index.strftime('%Y-%m-%d %H:%M:%S')
    day1.index = pd.to_datetime(day1.index)
    day1['Date'] = day1.index.date
    for i in range(len(day1.columns)):
        day1.columns.values[i] = "1d_" + day1.columns.values[i]
    day1.rename(columns={'1d_Date':'Date'}, inplace=True)
    day1 = day1.reindex(columns=['Date','1d_Open', '1d_High', '1d_Low', '1d_Close', '1d_Adj Close', '1d_Volume'])

    # Extend the index to the end of the day
    last_timestamp = day1.index[-1]
    end_of_day = last_timestamp.replace(hour=23, minute=59, second=59)
    new_index = pd.date_range(start=day1.index[0], end=end_of_day, freq='T')
    day1 = day1.reindex(new_index)

    day1 = day1.resample('5T').ffill()
    day1.fillna(method='ffill', inplace=True)


    # Concatenate dataframes
    main_df = pd.concat([min5, min15, min60, day1], axis=1)
    # Drop duplicate columns
    main_df = main_df.loc[:, ~main_df.columns.duplicated()]
    # pd.set_option('display.max_rows', None)
    main_df = main_df.dropna(subset=['5m_Open','5m_High','5m_Low','5m_Close'])
    
    return main_df

In [7]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Define an infinite loop to continuously fetch minute-level stock data and plot EMAs
while True:
    # # Download minute-level data for the stock index "Nifty 50" (^NSEI) starting from "2024-02-05"
    # df = yf.download("^NSEI", start="2024-02-25", interval="1m")

    df = fetch_data('^NSEBANK', '2024-04-10 09:15:00')

    df.index.rename("Datetime",inplace=True)

    # Reset index to make the 'Datetime' column available for plotting
    df.reset_index(inplace=True)

    df.fillna(method='ffill', inplace=True)

    # Call the MACD_plot function to plot Exponential Moving Averages (EMAs) on the downloaded data
    # Note: 'fig' is likely a pre-existing figure object used for plotting
    plot_ema = MACD_plot(data=df, fig=fig, short_window=9, large_window=13)

    print(df.tail(5))
    
    # Sleep for 1 minute before running the loop again                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              to avoid overwhelming the system and the API
    time.sleep(300)


                Datetime        Date       5m_Open       5m_High  \
2141 2024-05-24 12:40:00  2024-05-24  48864.050781  48867.351562   
2142 2024-05-24 12:45:00  2024-05-24  48858.398438  48875.199219   
2143 2024-05-24 12:50:00  2024-05-24  48851.898438  48877.000000   
2144 2024-05-24 12:55:00  2024-05-24  48861.398438  48932.101562   
2145 2024-05-24 13:00:00  2024-05-24  48916.851562  48923.148438   

            5m_Low      5m_Close  5m_Adj Close  5m_Volume      15m_Open  \
2141  48842.351562  48858.398438  48858.398438        0.0  48830.949219   
2142  48843.898438  48852.449219  48852.449219        0.0  48858.398438   
2143  48840.699219  48861.898438  48861.898438        0.0  48858.398438   
2144  48857.199219  48913.601562  48913.601562        0.0  48858.398438   
2145  48910.500000  48921.398438  48921.398438        0.0  48916.851562   

          15m_High       15m_Low     15m_Close  15m_Adj Close  15m_Volume  \
2141  48867.351562  48823.148438  48858.398438   48858.398438  